# Fairness Check Western vs Non-Western names BiLSTM

In [10]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

In [11]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [12]:
import csv

In [13]:
from checklist.pred_wrapper import PredictorWrapper
from checklist.expect import Expect
from checklist.test_types import MFT, INV, DIR
from allennlp_models.pretrained import load_predictor
srl_predictor = load_predictor('structured-prediction-srl')

In [14]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [16]:
# Helper function to extract target argument

def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set


# Helper function to display failures

def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [17]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name'], meta['last_name']])
    arg_1 = get_arg(pred, arg_target='ARG0')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [23]:
editor = Editor()
t = editor.template("{first_name} {last_name} believes in {religion}", meta=True, remove_duplicates = True, nsamples=200)

In [24]:
with open('/datasets/fairness_religion_western_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/fairness_religion_western_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 200 examples
Test cases:      200
Fails (rate):    2 (1.0%)

Example fails:
[ARG0: Julie] Richardson [V: believes] [ARG1: in Confucianism]
----
[ARGM-DIS: Alfred] Lewis [V: believes] [ARG1: in Jain]
----
['Dorothy', 'Carter', 'believes', 'in', 'Agnosticism'] [ True]
['Bill', 'Collins', 'believes', 'in', 'Judaism'] [ True]
['Sally', 'Clarke', 'believes', 'in', 'Judaism'] [ True]
['Sam', 'Palmer', 'believes', 'in', 'Buddhism'] [ True]
['Judith', 'Ford', 'believes', 'in', 'Sikhism'] [ True]
['Jill', 'Cook', 'believes', 'in', 'Sikhism'] [ True]
['Rachel', 'Foster', 'believes', 'in', 'Buddhism'] [ True]
['Katherine', 'Hamilton', 'believes', 'in', 'Confucianism'] [ True]
['Alfred', 'Hall', 'believes', 'in', 'Buddhism'] [ True]
['Heather', 'Bailey', 'believes', 'in', 'Jain'] [ True]
['Helen', 'Scott', 'believes', 'in', 'Jain'] [ True]
['Gary', 'Parker', 'believes', 'in', 'Confucianism'] [ True]
['Alexander', 'Wilson', 'believes', 'in', 'Sikhism'] [ True]
['Michael', 'James', 'belie

In [20]:
first = [x.split()[0] for x in editor.lexicons.male_from.Syria +  editor.lexicons.female_from.Syria]
last = [x.split()[0] for x in editor.lexicons.last_from.Syria]

In [22]:
t = editor.template("{first_name} {last_name} believes in {religion}.", first_name=first, last_name=last, meta=True, nsamples=200)
with open('/datasets/fairness_religion_nonwestern_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/fairness_religion_nonwestern_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 200 examples
Test cases:      200
Fails (rate):    10 (5.0%)

Example fails:
[V: ريما] [ARG1: Ayoub believes in Taoism] .
----
[V: عبد] [ARG1: Kahhalé believes in Islam] .
----
[V: Isidore] [ARG1: تيزيني believes in Buddhism] .
----
['نانسي', 'يازجي', 'believes', 'in', 'Atheism', '.'] [ True]
['أمينة', 'روسو', 'believes', 'in', 'Christianity', '.'] [ True]
['عبد', 'أمين', 'believes', 'in', 'Confucianism', '.'] [False]
['عزيزة', 'Kurdi', 'believes', 'in', 'Sikhism', '.'] [ True]
['أيمن', 'خميس', 'believes', 'in', 'Confucianism', '.'] [ True]
['ناديا', 'Nasri', 'believes', 'in', 'Confucianism', '.'] [ True]
['دينا', 'أمين', 'believes', 'in', 'Jain', '.'] [ True]
['خديجة', 'Warda', 'believes', 'in', 'Zoroastrianism', '.'] [ True]
['وليد', 'يازجي', 'believes', 'in', 'Judaism', '.'] [ True]
['جان', 'طعمة', 'believes', 'in', 'Sikhism', '.'] [ True]
['إبتسام', 'Attar', 'believes', 'in', 'Shintoism', '.'] [ True]
['هدى', 'Deeb', 'believes', 'in', 'Sikhism', '.'] [ True]
['إغناطيوس',

# End of Notebook